In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gliner

Spacy+Gliner

In [10]:
import spacy
import pandas as pd
import requests
import json
from gliner import GLiNER
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Carica il modello spaCy (ad esempio, 'en_core_web_sm')
nlp = spacy.load("en_core_web_sm")
"""
# Funzione per interrogare DBpedia Spotlight
def query_dbpedia_spotlight(text, confidence=0.5, support=20):
    spotlight_endpoint = "https://api.dbpedia-spotlight.org/en/annotate"
    headers = {"accept": "application/json"}
    params = {"text": text, "confidence": confidence, "support": support}
    response = requests.get(spotlight_endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
"""
# Funzione per interrogare DBpedia Spotlight
def query_dbpedia_spotlight(text, confidence=0.5, support=20):
    spotlight_endpoint = "https://api.dbpedia-spotlight.org/en/annotate"
    headers = {"accept": "application/json"}
    params = {"text": text, "confidence": confidence, "support": support}

    # Disable SSL certificate verification (use with caution!)
    response = requests.get(spotlight_endpoint, headers=headers, params=params, verify=False)

    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Funzione per estrarre categorie da DBpedia Spotlight
def extract_categories_from_spotlight(response):
    categories = {}
    if "Resources" in response:
        for resource in response["Resources"]:
            category = resource["@types"].split(",")[-1].split(":")[-1]
            if category not in categories:
                categories[category] = []
            categories[category].append(resource["@surfaceForm"])
        categories = {k: list(set(v)) for k, v in categories.items()}
    return categories

# Funzione per consolidare le categorie
def consolidate_categories_automatically(categories, frequency_threshold=1, similarity_threshold=0.4):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    category_names = [cat for cat in categories.keys() if cat != '']
    category_frequencies = {category: len(entities) for category, entities in categories.items()}
    filtered_categories = {k: v for k, v in categories.items() if k != '' and category_frequencies[k] >= frequency_threshold}
    if not filtered_categories:
        return {}

    category_names = list(filtered_categories.keys())
    category_embeddings = model.encode(category_names)
    similarity_matrix = cosine_similarity(category_embeddings)
    consolidated_categories = {}

    for i, category in enumerate(category_names):
        merged = False
        for j, target_category in enumerate(consolidated_categories.keys()):
            if similarity_matrix[i][j] > similarity_threshold:
                consolidated_categories[target_category].extend(filtered_categories[category])
                merged = True
                break
        if not merged:
            consolidated_categories[category] = filtered_categories[category]

    consolidated_categories = {k: list(set(v)) for k, v in consolidated_categories.items()}
    return consolidated_categories

# Funzione per utilizzare GLiNER per l'estrazione delle entità
def extract_entities_with_gliner(text, categories_file="categories_1.json"):
    with open(categories_file, "r") as file:
        categories = json.load(file)
    labels = list(categories.keys())
    labels = [label for label in labels if label]  # Rimuove etichette vuote
    print(f"Etichette utilizzate per l'estrazione: {labels}")

    model = GLiNER.from_pretrained("urchade/gliner_medium")
    entities = model.predict_entities(text, labels, threshold=0.4)

    extracted_entities = []
    for entity in entities:
        print(f"{entity['text']} => {entity['label']}")
        extracted_entities.append({"text": entity["text"], "label": entity["label"]})
    return extracted_entities

# Funzione per estrarre entità con spaCy
def extract_entities_with_spacy(text):
    doc = nlp(text)
    extracted_entities = []
    for ent in doc.ents:
        extracted_entities.append({"text": ent.text, "label": ent.label_})
    return extracted_entities

# Funzione per processare il dataset
def process_dataset(input_file, output_file, text_column="text"):
    df = pd.read_csv(input_file)
    all_results = []
    all_categories = {}  # Dizionario per accumulare tutte le categorie

    for index, row in df.iterrows():
        print(f"\nProcessing row {index + 1}/{len(df)}...")
        text = row[text_column]

        # Step 1: Interroga DBpedia Spotlight
        response = query_dbpedia_spotlight(text)

        # Step 2: Estrai e consolida le categorie
        categories = extract_categories_from_spotlight(response)
        consolidated_categories = consolidate_categories_automatically(categories)

        # Aggiorna il dizionario delle categorie
        for category, entities in consolidated_categories.items():
            if category not in all_categories:
                all_categories[category] = set()
            all_categories[category].update(entities)

        # Salva le categorie accumulate in categories_1.json
        save_categories_to_file(all_categories)

        # Step 3: Estrai entità con GLiNER
        extracted_entities = extract_entities_with_gliner(text)

        # Step 4: Estrai entità con spaCy
        spacy_entities = extract_entities_with_spacy(text)

        # Adatta il formato di output di spaCy a quello di GLiNER
        spacy_entities_adapted = []
        for entity in spacy_entities:
            spacy_entities_adapted.append({
                "text": entity["text"],
                "label": entity["label"]  # Utilizza 'label' come chiave per uniformità
            })

        # Confronta i risultati di GLiNER e spaCy (adattato)
        print("\nConfronto tra GLiNER e spaCy:")
        print("Entità estratte da GLiNER:", extracted_entities)
        print("Entità estratte da spaCy (adattato):", spacy_entities_adapted)

        # Salva i risultati per ogni riga (inclusi i risultati di spaCy)
        all_results.append({
            "id": row["id"],
            "text": text,
            "entities_gliner": extracted_entities,
            "entities_spacy": spacy_entities_adapted  # Aggiungi i risultati di spaCy
        })

# Salva tutti i risultati in un file JSON
    with open(output_file, "w") as file:
        json.dump(all_results, file, indent=4)
    print(f"File '{output_file}' creato con successo!")

def save_categories_to_file(categories, output_file="categories_1.json"):
    # Converti i set in liste per la serializzazione JSON
    categories_to_save = {k: list(v) for k, v in categories.items()}

    # Sovrascrivi il file con le categorie aggiornate
    with open(output_file, "w") as file:
        json.dump(categories_to_save, file, indent=4)
    print(f"Categorie salvate in '{output_file}'")

# Esecuzione della pipeline sul dataset
input_file = "dataset_example.csv"  # Dataset di esempio generato
output_file = "entities_output_2.json"  # File di output con le entità estratte
process_dataset(input_file, output_file)


Processing row 1/50...


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.dbpedia-spotlight.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Categorie salvate in 'categories_1.json'
Etichette utilizzate per l'estrazione: ['Person', 'Company', 'Rocket', 'SoccerClub']


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.dbpedia-spotlight.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Categorie salvate in 'categories_1.json'
Etichette utilizzate per l'estrazione: ['Person', 'Company', 'Rocket', 'SoccerClub', 'Museum', 'Settlement', 'Organisation', 'Astronaut', 'SpaceMission', 'Scientist', 'Deity', 'Planet', 'Convention', 'Language', 'Country', 'City', 'Artist', 'SoccerLeague', 'InformationAppliance', 'Software', 'FilmFestival', 'NationalFootballLeagueEvent', 'Mountain', 'AdministrativeRegion', 'Website', 'Venue', 'Village', 'Award', 'Town', 'MusicGenre', 'Station', 'SpaceStation', 'Automobile', 'Politician']


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


World Health Organization => Organisation
WHO => Organisation
campagne globali di salute pubblica => Convention

Confronto tra GLiNER e spaCy:
Entità estratte da GLiNER: [{'text': 'World Health Organization', 'label': 'Organisation'}, {'text': 'WHO', 'label': 'Organisation'}, {'text': 'campagne globali di salute pubblica', 'label': 'Convention'}]
Entità estratte da spaCy (adattato): [{'text': 'La World Health Organization', 'label': 'ORG'}]
File 'entities_output_2.json' creato con successo!
